In [120]:
import pandas as pd

In [121]:
# import the dataset
import pandas as pd
problems_df = pd.read_csv('ppp.csv')
target_df = pd.read_csv('tar.csv')

In [122]:
print('The dimensions of movies dataframe are:', problems_df.shape,'\nThe dimensions of ratings dataframe are:', target_df.shape)

The dimensions of movies dataframe are: (20114, 12) 
The dimensions of ratings dataframe are: (8270, 3)


In [123]:
problems_df.drop(['Submission_Id','User_Rating'], axis=1, inplace=True)
problems_df.drop(['Problem_Rating','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'],axis=1,inplace=True)
result_df = problems_df.drop_duplicates(subset=['Problem Name'], keep='first')

In [124]:
print('The dimensions of movies dataframe are:', problems_df.shape,'\nThe dimensions of ratings dataframe are:', target_df.shape)


The dimensions of movies dataframe are: (20114, 4) 
The dimensions of ratings dataframe are: (8270, 3)


In [125]:
# Take a look at movies_df
problems_df.head()

,prob_id,Problem Name,User_Name,Tags
0,1,1037CEqualize,akshayjadhav7670,"['dp', 'greedy', 'strings']"
1,2,266BQueue at the School,aryans_18,"['constructive algorithms', 'graph matchings',..."
2,3,1583BOmkar and Heavenly Tree,abhishek_s3107,"['constructive algorithms', 'trees']"
3,4,112APetya and Strings,fjafari,"['implementation', 'strings']"
4,5,443AAnton and Letters,manish_rohila,"['constructive algorithms', 'implementation']"


In [126]:
target_df['TargetVariable'] = 10* target_df['TargetVariable']

In [127]:
target_df.head()

,tar_id,User_ID,TargetVariable
0,1,1,2.558140
1,2,1,3.538462
2,3,1,4.000000
3,4,1,2.615385
4,5,1,4.736842


In [128]:
# Movie ID to movie name mapping
problem_name = problems_df.set_index('prob_id')['Problem Name'].to_dict()

n_probs = len(problems_df.prob_id.unique())#not sure whats happening here  #n_user changed to n_probs

n_items = len(target_df.tar_id.unique())
print("Number of unique users:", n_items)
print("Number of unique problems:", n_probs)
print("The full rating matrix will have:", n_probs*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(target_df))
print("Therefore: ", len(target_df) / (n_probs*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 8270
Number of unique problems: 20114
The full rating matrix will have: 166342780 elements.
----------
Number of ratings: 8270
Therefore:  0.004971661529283087 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [129]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        #users = data[:,0]
        #print("Users: ",users)
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [130]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = target_df.copy()#changed ratings to target
        
        # Extract all user IDs and movie IDs
        users = target_df.tar_id.unique()      ###changine user_id to tar_id
        movies = target_df.tar_id.unique()      ##i changed somethng
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.tar_id = target_df.tar_id.apply(lambda x: self.movieid2idx[x])
        self.ratings.tar_id = target_df.tar_id.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['TargetVariable'], axis=1).values
        self.y = self.ratings['TargetVariable'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [131]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_probs, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(20114, 8)
  (item_factors): Embedding(8270, 8)
)
user_factors.weight tensor([[0.0258, 0.0254, 0.0470,  ..., 0.0166, 0.0351, 0.0372],
        [0.0020, 0.0379, 0.0305,  ..., 0.0405, 0.0244, 0.0449],
        [0.0481, 0.0455, 0.0187,  ..., 0.0417, 0.0186, 0.0283],
        ...,
        [0.0445, 0.0235, 0.0447,  ..., 0.0030, 0.0122, 0.0469],
        [0.0399, 0.0161, 0.0317,  ..., 0.0023, 0.0098, 0.0266],
        [0.0401, 0.0435, 0.0430,  ..., 0.0317, 0.0230, 0.0140]])
item_factors.weight tensor([[0.0091, 0.0303, 0.0481,  ..., 0.0172, 0.0451, 0.0373],
        [0.0075, 0.0144, 0.0316,  ..., 0.0215, 0.0332, 0.0278],
        [0.0374, 0.0387, 0.0200,  ..., 0.0244, 0.0134, 0.0033],
        ...,
        [0.0096, 0.0178, 0.0375,  ..., 0.0322, 0.0389, 0.0474],
        [0.0321, 0.0438, 0.0468,  ..., 0.0289, 0.0307, 0.0468],
        [0.0014, 0.0440, 0.0241,  ..., 0.0430, 0.0065, 0.0030]])


In [132]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

/tmp/ipykernel_5465/59576823.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 20.042263999352087
iter #1 Loss: 20.01335082420936
iter #2 Loss: 19.95858550438514
iter #3 Loss: 19.93229475754958
iter #4 Loss: 19.848736190795897
iter #5 Loss: 19.761001088069037
iter #6 Loss: 19.653706770676834
iter #7 Loss: 19.53711797274076
iter #8 Loss: 19.39386869577261
iter #9 Loss: 19.233802766066333
iter #10 Loss: 19.06382654630221
iter #11 Loss: 18.87173796433669
iter #12 Loss: 18.636272694514346
iter #13 Loss: 18.419211666400617
iter #14 Loss: 18.186318646944486
iter #15 Loss: 17.933418919489934
iter #16 Loss: 17.66324261885423
iter #17 Loss: 17.381022321260893
iter #18 Loss: 17.088177666297327
iter #19 Loss: 16.79549692594088
iter #20 Loss: 16.499397424551155
iter #21 Loss: 16.1632602251493
iter #22 Loss: 15.83986599261944
iter #23 Loss: 15.522537480867825
iter #24 Loss: 15.190907947833722
iter #25 Loss: 14.845280691293569
iter #26 Loss: 14.486412796607384
iter #27 Loss: 14.128478226294884
iter #28 Loss: 13.79577991779034
iter #29 Loss: 13.440299356900729
ite

In [133]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.2321, 0.2292, 0.2462,  ..., 0.2181, 0.2336, 0.2380],
        [0.3099, 0.3430, 0.3293,  ..., 0.3430, 0.3227, 0.3462],
        [0.3871, 0.3811, 0.3481,  ..., 0.3751, 0.3475, 0.3602],
        ...,
        [0.0445, 0.0235, 0.0447,  ..., 0.0030, 0.0122, 0.0469],
        [0.0399, 0.0161, 0.0317,  ..., 0.0023, 0.0098, 0.0266],
        [0.0401, 0.0435, 0.0430,  ..., 0.0317, 0.0230, 0.0140]],
       device='cuda:0')
item_factors.weight tensor([[9.1464e-03, 3.0326e-02, 4.8137e-02,  ..., 1.7189e-02, 4.5065e-02,
         3.7301e-02],
        [1.3298e+00, 1.3602e+00, 1.3911e+00,  ..., 1.3493e+00, 1.3768e+00,
         1.3805e+00],
        [7.5468e-01, 7.2884e-01, 7.2583e-01,  ..., 7.0469e-01, 7.0961e-01,
         6.7983e-01],
        ...,
        [9.6250e-03, 1.7837e-02, 3.7526e-02,  ..., 3.2151e-02, 3.8916e-02,
         4.7398e-02],
        [3.2081e-02, 4.3751e-02, 4.6777e-02,  ..., 2.8914e-02, 3.0730e-02,
         4.6779e-02],
        [1.3634e-03, 4.4032e-02, 2.4109e

In [134]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [135]:
len(trained_movie_embeddings) # unique movie factor weights

8270

In [136]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [139]:
'''m is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorith
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = target_df.loc[target_df['User_ID']==movid].count()[0]
    movs.append((problem_name[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 112APetya and Strings
	 513G2Inversions problem
	 1521ANastia and Nearly Good Numbers
	 236ABoy or Girl
	 1480CSearching Local Minimum
	 1592AGamer Hemose
	 1480CSearching Local Minimum
	 124AThe number of positions
	 676CVasya and String
	 312BArcher
Cluster #1
	 616BDinner with Emma
	 1642CGreat Sequence
	 1398ABad Triangle
	 1578EEasy Scheduling
	 1335ACandies and Two Sisters
	 771ABear and Friendship Condition
	 932BRecursive Queries
	 1556BTake Your Places!
	 550BPreparing Olympiad
	 371CHamburgers
Cluster #2
	 1037CEqualize
	 496AMinimum Difficulty
	 977AWrong Subtraction
	 143AHelp Vasilisa the Wise 2
	 1562BScenes From a Memory
	 449BJzzhu and Cities
	 1633CKill the Monster
	 1644ADoors and Keys
	 1271ASuits
	 1037CEqualize
Cluster #3
	 1583BOmkar and Heavenly Tree
	 189ACut Ribbon
	 1406BMaximum Product
	 268BButtons
	 556ACase of the Zeros and Ones
	 1637CAndrew and Stones
	 996AHit the Lottery
	 339AHelpful Maths
	 580AKefa and First Steps
	 158ANext Round
Clust